<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
#make the data

import numpy as np

np.random.seed(812)

X = np.array(([0, 0, 1],
             [0, 1, 1],
             [1, 0, 1],
             [0, 1, 0],
             [1, 0, 0],
             [1, 1, 1],
             [0, 0, 0]), dtype=float)

y = np.array(([0],
             [1],
             [1],
             [1],
             [1],
             [0],
             [0]), dtype=float)

In [2]:
X, y

(array([[0., 0., 1.],
        [0., 1., 1.],
        [1., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.]]), array([[0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.]]))

In [7]:
#we don't need to normalize the features as they are all zeros and ones

class NeuralNetwork:
    
    def __init__(self):
        #creating the architechture 
        
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        #create the weights from input to hidden, should be 3x4 shape
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        
        #create weights from hidden to output should be 4x1
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    #we will need this for activation    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    #this is the derivitave we will use for the gradient descent 
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    #this can create an initial prediction, its the nn without training
    def feed_forward(self, X):
        
        #create weighted sum for first layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        #activate hidden layer with the sigmoid function
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        #weighted sum of hidden layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #activation of output layer
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    #now we will add the back propogation, which allows the nn to be trained
    def backward(self, X, y, o):
        
        #this will give us the absolute error of the output
        self.o_error = y - o
        
        #this gives us the derivitave of the error, which will allow us to find out how much the weights are off
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        #with that, how much were our output layer weights off?
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        #and how much were our hidden layer weights off?
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        #now we can adjust the first weights, from input to hidden
        self.weights1 += X.T.dot(self.z2_delta)
        
        #and adjust our second weights, from hidden to output
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [8]:
nn = NeuralNetwork()

nn.train(X, y)

In [10]:
#output is pretty far off from the OG
nn.activated_output, y

(array([[0.5388686 ],
        [0.55352047],
        [0.40771131],
        [0.55447957],
        [0.3955229 ],
        [0.42425304],
        [0.53465072]]), array([[0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.]]))

In [11]:
#this is how much of a difference there was from the activated output to y 
nn.o_error

array([[-0.5388686 ],
       [ 0.44647953],
       [ 0.59228869],
       [ 0.44552043],
       [ 0.6044771 ],
       [-0.42425304],
       [-0.53465072]])

In [12]:
#this is how much the hidden to output layer weights were responsible for that
nn.o_delta

array([[-0.12539193],
       [ 0.10348858],
       [ 0.14208527],
       [ 0.10323952],
       [ 0.14535981],
       [-0.10143027],
       [-0.12454807]])

In [13]:
#this is how much the input to the hidden layer weights were responsible for that 
nn.z2_delta

array([[-0.00759937,  0.01187734,  0.01762239, -0.03003046],
       [ 0.00629603, -0.00988339, -0.01392096,  0.02401908],
       [ 0.00865308, -0.01350147, -0.01872003,  0.03655789],
       [ 0.00613109, -0.00974512, -0.0144451 ,  0.02444225],
       [ 0.00878458, -0.01325518, -0.01985593,  0.03751597],
       [-0.00617866,  0.00969073,  0.0129677 , -0.02588203],
       [-0.0071323 ,  0.01119298,  0.01818608, -0.0303734 ]])

In [14]:
#lets for real train the net

nn = NeuralNetwork()

for i in range(10000):
    if (i +1 in [1,2,3,4,5]) or ((i +1) % 1000 == 0):
        print ('+' + '___' *3 + f'EPOCH {i +1}' + '___'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print('Loss: \n', str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

+_________EPOCH 1_________+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.75008706]
 [0.80117548]
 [0.80603981]
 [0.81941677]
 [0.82355324]
 [0.8420462 ]
 [0.76931163]]
Loss: 
 0.286344042468969
+_________EPOCH 2_________+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.66050711]
 [0.72445107]
 [0.72147377]
 [0.75855708]
 [0.75569123]
 [0.77333451]
 [0.69710605]]
Loss: 
 0.2559654702338422
+_________EPOCH 3_________+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
 [[0.59112974]
 [0.66505987]
 [0.65445359]
 [0.71151264]
 [0.7018249 ]
 [0.71887594]
 [0.64058972]]
Loss: 
 0.24004181848412465
+_________EPOCH 

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

In [2]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [5]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [6]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [7]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [8]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [ ]:
##### Your Code Here #####



## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?